# Flavor Intelligence Review

Interactive walkthrough of the analytics pipeline against the 38.8K-row backfill dataset.
Run cells top-to-bottom to explore each phase.

In [ ]:
import sys, os
# Ensure the project root is on the path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..' if os.path.basename(os.getcwd()) == 'analytics' else '.'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 5)

from analytics.data_loader import load_clean, flavor_list, store_list
df = load_clean()
print(f'{len(df):,} observations | {df["store_slug"].nunique()} stores | {df["title"].nunique()} flavors')
print(f'Date range: {df["flavor_date"].min().date()} to {df["flavor_date"].max().date()}')

## Phase 1: Basic Metrics

In [ ]:
from analytics.basic_metrics import (
    flavor_frequency, flavor_probability, days_since_last,
    overdue_flavors, shannon_entropy, pielou_evenness,
    surprise_score, store_summary
)

# Global flavor frequency
freq = flavor_frequency(df)
fig, ax = plt.subplots(figsize=(14, 6))
freq.plot.barh(ax=ax, color='#1f77b4')
ax.set_xlabel('Total appearances across all stores')
ax.set_title('Global Flavor Frequency (all 154 WI stores)')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Mt. Horeb store summary
summary = store_summary(df, 'mt-horeb')
print('=== Mt. Horeb Store Summary ===')
for k, v in summary.items():
    print(f'  {k}: {v}')

In [ ]:
# Overdue flavors at mt-horeb
overdue = overdue_flavors(df, 'mt-horeb')
if len(overdue) > 0:
    print(f'{len(overdue)} overdue flavors at Mt. Horeb:\n')
    print(overdue.head(10).to_string(index=False))
else:
    print('No overdue flavors at Mt. Horeb')

In [ ]:
# Surprise scores — most and least surprising flavors at mt-horeb
probs = flavor_probability(df, 'mt-horeb')
surprises = pd.Series({f: surprise_score(df, 'mt-horeb', f) for f in probs.index})
surprises = surprises[surprises < float('inf')].sort_values()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
surprises.head(10).plot.barh(ax=axes[0], color='green')
axes[0].set_title('Least surprising (most common)')
axes[0].set_xlabel('Surprise (bits)')
axes[0].invert_yaxis()

surprises.tail(10).plot.barh(ax=axes[1], color='red')
axes[1].set_title('Most surprising (rarest)')
axes[1].set_xlabel('Surprise (bits)')
axes[1].invert_yaxis()
plt.suptitle('Flavor Surprise at Mt. Horeb: -log\u2082(P(flavor|store))', fontsize=13)
plt.tight_layout()
plt.show()

## Phase 2: Pattern Detection

In [ ]:
from analytics.patterns import dow_chi_squared, recurrence_intervals, seasonal_heatmap, seasonal_flavors

# Day-of-week bias
dow_results = dow_chi_squared(df)
significant = dow_results[dow_results['p_value'] < 0.05]
print(f'{len(significant)} flavors with significant day-of-week bias (p < 0.05):\n')
print(significant[['title', 'chi2', 'p_value', 'peak_name']].head(15).to_string(index=False))

In [ ]:
# Recurrence intervals — the rotation cycle
intervals = recurrence_intervals(df)

fig, ax = plt.subplots(figsize=(12, 6))
intervals.set_index('title')['mean_gap'].sort_values().plot.barh(ax=ax, color='#2ca02c')
ax.axvline(x=intervals['mean_gap'].median(), color='red', linestyle='--',
           label=f'Median: {intervals["mean_gap"].median():.0f} days')
ax.set_xlabel('Mean days between repeat servings')
ax.set_title('Flavor Recurrence Intervals (across all stores)')
ax.legend()
ax.invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Seasonal heatmap
heatmap = seasonal_heatmap(df)
month_labels = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
heatmap_norm = heatmap.div(heatmap.sum(axis=1), axis=0)

fig, ax = plt.subplots(figsize=(14, 10))
im = ax.imshow(heatmap_norm.values, aspect='auto', cmap='YlOrRd')
ax.set_xticks(range(12))
ax.set_xticklabels(month_labels)
ax.set_yticks(range(len(heatmap_norm)))
ax.set_yticklabels(heatmap_norm.index, fontsize=8)
ax.set_title('Seasonal Heatmap: Flavor x Month (row-normalized)')
plt.colorbar(im, ax=ax, label='Relative frequency')
plt.tight_layout()
plt.show()

seasonal = seasonal_flavors(df)
if len(seasonal) > 0:
    print('\nSeasonal flavors (>50% in a 3-month window):\n')
    print(seasonal.to_string(index=False))

In [ ]:
from analytics.markov import transition_matrix, top_transitions, self_transition_rate

tm = transition_matrix(df)

fig, ax = plt.subplots(figsize=(14, 12))
im = ax.imshow(tm.values, cmap='Blues', vmin=0, vmax=0.1)
ax.set_xticks(range(len(tm.columns)))
ax.set_xticklabels(tm.columns, rotation=90, fontsize=7)
ax.set_yticks(range(len(tm.index)))
ax.set_yticklabels(tm.index, fontsize=7)
ax.set_xlabel("Next day's flavor")
ax.set_ylabel("Today's flavor")
ax.set_title('Markov Transition Matrix: P(tomorrow | today)')
plt.colorbar(im, ax=ax, label='Transition probability')
plt.tight_layout()
plt.show()

print('\nMost likely flavors after Turtle:')
for t in top_transitions(df, 'Turtle', n=5):
    print(f"  {t['flavor']}: {t['probability']:.1%}")

## Phase 3: Collaborative Filtering

In [ ]:
from analytics.collaborative import (
    store_flavor_matrix, nmf_decompose, factor_top_flavors,
    cluster_stores, cluster_summary, cluster_geo_summary, load_store_locations
)

matrix = store_flavor_matrix(df, normalize_rows=True)
W, H, model = nmf_decompose(matrix, n_components=6)

tops = factor_top_flavors(H, n=5)
print('NMF Latent Factors — top 5 flavors each:\n')
for factor, flavors in tops.items():
    print(f'  {factor}: {", ".join(flavors)}')

In [ ]:
clusters = cluster_stores(W, n_clusters=5)
summary_c = cluster_summary(df, clusters)

print('Store Clusters:\n')
for cid, info in summary_c.items():
    print(f'  Cluster {cid}: {info["size"]} stores')
    print(f'    Top flavors: {", ".join(info["top_flavors"])}')
    print(f'    Examples: {", ".join(info["example_stores"][:3])}')
    print()

In [ ]:
# Geographic cluster scatter
locs = load_store_locations()
cluster_df = clusters.rename('cluster').reset_index()
cluster_df.columns = ['slug', 'cluster']
merged = locs.merge(cluster_df, on='slug', how='inner')

fig, ax = plt.subplots(figsize=(12, 7))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
for cid in sorted(merged['cluster'].unique()):
    c = merged[merged['cluster'] == cid]
    ax.scatter(c['lng'], c['lat'], c=colors[cid], label=f'Cluster {cid} ({len(c)})', alpha=0.7, s=20)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Store Clusters by Geographic Location (WI stores)')
ax.legend()
plt.tight_layout()
plt.show()

print('\nGeographic summary:')
print(cluster_geo_summary(clusters).to_string(index=False))

## Phase 4: Prediction Models

In [ ]:
from analytics.predict import FrequencyRecencyModel, MarkovRecencyModel
from analytics.evaluate import time_split, evaluate_model, compare_models

train, test = time_split(df, '2026-01-01')
print(f'Train: {len(train):,} rows (pre-2026) | Test: {len(test):,} rows (2026+)\n')

freq_model = FrequencyRecencyModel().fit(train)
markov_model = MarkovRecencyModel().fit(train)

comparison = compare_models(
    {'Frequency+Recency': freq_model, 'Markov+Recency': markov_model},
    test, max_samples=500
)
print('Model Comparison (500 test samples):\n')
print(comparison.round(4).to_string())
print(f'\nRandom baseline: top_1 = {1/df["title"].nunique():.4f}, top_5 = {5/df["title"].nunique():.4f}')

In [ ]:
# Prediction distribution for mt-horeb
proba = freq_model.predict_proba('mt-horeb', pd.Timestamp('2026-02-23'))
top15 = proba.nlargest(15)

fig, ax = plt.subplots(figsize=(10, 6))
top15.sort_values().plot.barh(ax=ax, color='#ff7f0e')
ax.set_xlabel('Predicted probability')
ax.set_title('Flavor Forecast: Mt. Horeb, 2026-02-23 (top 15)')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

## Phase 5: Embeddings & Forecast

In [ ]:
from analytics.embeddings import SEED_CATALOG, nearest_neighbors, validate_against_groups
from sklearn.feature_extraction.text import TfidfVectorizer

texts = [f"{f['title']}: {f['description']}" for f in SEED_CATALOG]
titles = [f['title'] for f in SEED_CATALOG]
embeddings = TfidfVectorizer().fit_transform(texts).toarray().astype(np.float32)

print('Nearest neighbors for "Turtle":')
for n in nearest_neighbors(embeddings, titles, 'Turtle', n=5):
    print(f"  {n['title']}: {n['similarity']:.3f}")

print('\nNearest neighbors for "Mint Cookie":')
for n in nearest_neighbors(embeddings, titles, 'Mint Cookie', n=5):
    print(f"  {n['title']}: {n['similarity']:.3f}")

In [ ]:
# 2D projection of flavor embeddings
from sklearn.manifold import TSNE

proj = TSNE(n_components=2, random_state=42, perplexity=8).fit_transform(embeddings)

fig, ax = plt.subplots(figsize=(12, 8))
ax.scatter(proj[:, 0], proj[:, 1], s=60, alpha=0.7)
for i, title in enumerate(titles):
    ax.annotate(title, (proj[i, 0], proj[i, 1]), fontsize=7, ha='center', va='bottom')
ax.set_title('Flavor Embedding Space (TF-IDF + t-SNE)')
ax.set_xlabel('t-SNE 1')
ax.set_ylabel('t-SNE 2')
plt.tight_layout()
plt.show()

In [ ]:
from analytics.forecast_writer import build_forecast_context, format_forecast_template

ctx = build_forecast_context(freq_model, df, 'mt-horeb', pd.Timestamp('2026-02-23'))
print(format_forecast_template(ctx))

## Store Diversity Overview

In [ ]:
store_metrics = [store_summary(df, slug) for slug in store_list(df)]
sm = pd.DataFrame(store_metrics)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
sm['entropy'].hist(ax=axes[0], bins=20, color='#2ca02c', edgecolor='white')
axes[0].set_xlabel('Shannon Entropy (bits)')
axes[0].set_title(f'Store Entropy Distribution (n={len(sm)})')
axes[0].axvline(sm['entropy'].median(), color='red', linestyle='--',
                label=f'Median: {sm["entropy"].median():.2f}')
axes[0].legend()

sm['evenness'].hist(ax=axes[1], bins=20, color='#9467bd', edgecolor='white')
axes[1].set_xlabel("Pielou's Evenness J (0-1)")
axes[1].set_title('Store Evenness Distribution')
axes[1].axvline(sm['evenness'].median(), color='red', linestyle='--',
                label=f'Median: {sm["evenness"].median():.2f}')
axes[1].legend()
plt.tight_layout()
plt.show()